## Admission Management System in Pyhton

In [1]:
#importing required libraries
import pandas as pd
import csv
import numpy as np  
import random as rand

## Section 1. Generate the college_details.csv and studentdetails.csv file. Make atleast 50 college entries and 5000 students data.

### Note:

#### i) There will be around 25 files named as day(no)\_college\_status.csv because there are 5000 student and 200 students get admission on each day so (5000/200)=25.

#### ii)Almost all the student will get admission in Section (3&4). Because there is 50 college, each with 9 branch and intake ranging from 0 to 240 but threre are only 5000 students. So students will get accomodated. But if we wish to test the system then increase no. of student or decrease no. of colleges. which can be done from next 2 cells.

In [30]:
#Create College_details.csv
seats_list=[0,60,120,180,240]
first_row=['CollegeCode','CSE','IT','ME','EE','EC','IC','CH','CIVIL','AUTO']#branches
file1="College_details.csv"
with open(file1,'w') as csvfile1: #opening a file
    csvwrt1=csv.writer(csvfile1)
    csvwrt1.writerow(first_row)
    for i in range(1,51):        #50 colleges data
        push_row=['C{:04}'.format(i),rand.choice(seats_list),rand.choice(seats_list),rand.choice(seats_list),rand.choice(seats_list),rand.choice(seats_list),rand.choice(seats_list),rand.choice(seats_list),rand.choice(seats_list),rand.choice(seats_list)]
        csvwrt1.writerow(push_row)


In [31]:
#Create studentdetails.csv
first_row=['Enrollmentno','sub1','sub2','sub3','sub4','sub5']  #creating file columns structure
file2="studentdetails.csv"
with open(file2,'w') as csvfile2:
    csvwrt2=csv.writer(csvfile2)
    csvwrt2.writerow(first_row)
    for i in range(1,5001):   #5000 students data
        push_row=['E{:04}'.format(i),rand.randint(40,100),rand.randint(40,100),rand.randint(40,100),rand.randint(40,100),rand.randint(40,100)]
        csvwrt2.writerow(push_row)


## Section 2. Give rank to the students so that admission process will be performed in proper sequence. Use any mechanism. Here I have used total of all 5 subjects for rank and if the total is same than rank based on sub1 then sub2 upto sub 5. 

In [32]:
clg_data=pd.read_csv('College_details.csv')  #reading files
std_data=pd.read_csv('studentdetails.csv')
possible_fields=[['CSE','IT']    #creating group of branch
                ,['ME','IC','AUTO']
                ,['EE','EC']
                ,['CIVIL','CH']]
field_list=['CSE','IT','ME','EE','EC','IC','CH','CIVIL','AUTO']
nonzero_dict=dict()    #creating dictonary with the collges having nonzero intake
for i in field_list:
    nonzero_dict[i]=list(clg_data['CollegeCode'].iloc[np.nonzero(list(clg_data[i]))])

### Below code generates 'choice_filling.csv' which contains Enrollment No and 10 choices for each students and the choices are based on branch groups means if student applies for CSE then he/she can also choose IT but not ME,similarly EE and EC are same group. This is just to make the data a little bit more accurate.

In [33]:
file3="choice_filling.csv"
with open(file3,'w') as csvfile3:
    csvwrt3=csv.writer(csvfile3)
    first_row=['Enrollmentno','choice1','choice2','choice3','choice4','choice5','choice6','choice7','choice8','choice9','choice10']
    csvwrt3.writerow(first_row) #creating choice_filling.csv for 10 choices
    for i in range(len(list(std_data['Enrollmentno']))):
        temp_fields=rand.choice(possible_fields)
        enno=std_data['Enrollmentno'][i]
        filled_list=[enno]
        for j in range(10):
            sub=rand.choice(temp_fields)
            filled_list.append(str(rand.choice(nonzero_dict[sub])+'-'+sub))
        csvwrt3.writerow(filled_list)


### Below code will sort the data and give rank to each student and store it in 'rank_student.csv'

In [34]:
total_marks=list()
for i in range(len(std_data['Enrollmentno'])):  #making total of all students and sorting accordingly
    total_marks.append(std_data['sub1'][i]+std_data['sub2'][i]+std_data['sub3'][i]+std_data['sub4'][i]+std_data['sub5'][i])
std_data.insert(6,'total',total_marks)        

sortData=std_data.sort_values(['total','sub1','sub2','sub3','sub4','sub5'],ascending=False)
sortData['Enrollmentno'].to_csv('rank_student.csv',index=False)

## Section (3 and 4): The following code will give admission to 200 students on each day in their preferable collge and generate 'day1_college_status.csv' which has details of available seats in all the colleges for admission on next day.

In [35]:
std_marks_data=pd.read_csv('studentdetails.csv')  #reads student details 
not_admitted=pd.DataFrame(columns=std_marks_data.columns) #creates blank schema for not admitted students
clg_data=pd.read_csv('College_details.csv')  #reads clg data
filled_seats=pd.DataFrame(columns=clg_data.columns,index=clg_data.index)
filled_seats['CollegeCode']=clg_data['CollegeCode']
filled_seats.fillna(0,inplace=True)
remaining_seats=clg_data
sortData=pd.read_csv('rank_student.csv')  #reads rank details
admitted_students=pd.DataFrame(columns=['Enrollmentno','CollegeCode','Branch','Round'])  #creates blank schema for admitted students
admission_count=0

day_round=1
total_students=sortData.index.size
student_index_count=1
# total_students=1000
choice_data=pd.read_csv('choice_filling.csv')

while (student_index_count <= total_students):  #loop until all the students are not processed
  
    for i in range(200):    #on each day only 200 students get admission
        tempstd=sortData['Enrollmentno'][i]
        student_index_count+=1
        temp_choice=choice_data.loc[choice_data['Enrollmentno']==tempstd]
        for j in range(10):  # loop through all 10 choices
            clg,branch=temp_choice['choice'+str(j+1)][int(tempstd[1:5])-1].split('-')  
    #         print(clg,branch)
            if int(remaining_seats.loc[remaining_seats['CollegeCode']==clg][branch])>0: #check if the college has any seats available
                remaining_seats.loc[remaining_seats.CollegeCode==clg,branch]-=1
                filled_seats.loc[filled_seats.CollegeCode==clg,branch]+=1
                admitted_students.loc[admission_count]=[tempstd,clg,branch,day_round]
                admission_count+=1
                break
            elif j==9:  #no college is vacant according to this student's choice so he wont get admission 
                enroll=int(tempstd[1:4])-1
                not_admitted=not_admitted.append({'Enrollmentno':tempstd,'sub1':std_marks_data['sub1'][enroll],'sub2':std_marks_data['sub2'][enroll],'sub3':std_marks_data['sub3'][enroll],'sub4':std_marks_data['sub4'][enroll],'sub5':std_marks_data['sub5'][enroll]},ignore_index=True)
               
            else:
                #he/she didn't get admission for this choice but can get admission in next remaining choices
                pass
    remaining_seats.to_csv('day'+str(day_round)+'_college_status.csv',index=False)  #creates day1_college_status.csv on each day having remaining choice details for all the collges
    day_round+=1   #increase day 
remaining_seats.to_csv('Remaining_Seats.csv',index=False)    #remaining seats after whole admission process is over
not_admitted.to_csv('Not_Admitted_Students.csv',index=False) #students who did not got admission

## Section (5 and 6): If any student did not get any admission then generate the choice details one more time based on available seats and once the admission process is complete give the status of colleges and show in table how many seat remain vacant in each branch of college.

In [36]:
clg_data_remaining=pd.read_csv('Remaining_Seats.csv')   #repeating the above process again who are not admitted
std_not_admitted_data=pd.read_csv('Not_Admitted_Students.csv')
possible_fields=[['CSE','IT']
                ,['ME','IC','AUTO']
                ,['EE','EC']
                ,['CIVIL','CH']]
field_list=['CSE','IT','ME','EE','EC','IC','CH','CIVIL','AUTO']
nonzero_dict=dict()
for i in field_list:
    nonzero_dict[i]=list(clg_data_remaining['CollegeCode'].iloc[np.nonzero(list(clg_data_remaining[i]))])
    

In [37]:
file7="choice_filling_not_admitted.csv"   #making total of subjects for sorting and giving rank to students
with open(file7,'w') as csvfile7:
    csvwrt7=csv.writer(csvfile7)
    first_row=['Enrollmentno','choice1','choice2','choice3','choice4','choice5','choice6','choice7','choice8','choice9','choice10']
    csvwrt7.writerow(first_row)
    for i in range(len(list(std_not_admitted_data['Enrollmentno']))):
        temp_fields=rand.choice(possible_fields)
        enno=std_not_admitted_data['Enrollmentno'][i]
        filled_list=[enno]
        for j in range(10):
            sub=rand.choice(temp_fields)
            if len(nonzero_dict[sub])>0:
                filled_list.append(str(rand.choice(nonzero_dict[sub])+'-'+sub))
        csvwrt7.writerow(filled_list)

In [38]:
total_marks=list()
for i in range(len(not_admitted['Enrollmentno'])):
    total_marks.append(not_admitted['sub1'][i]+not_admitted['sub2'][i]+not_admitted['sub3'][i]+not_admitted['sub4'][i]+not_admitted['sub5'][i])
not_admitted.insert(6,'total',total_marks)        

sortData=not_admitted.sort_values(['total','sub1','sub2','sub3','sub4','sub5'],ascending=False)

sortData['Enrollmentno'].to_csv('rank_not_admitted.csv',index=False)


### Below code works almost same like previous (3&4) section code as it does the admission process of those student who did not get admission in any college. It retrieves student data from not_admitted dataframe and use it with 'Remaining_Seats.csv' to check which college has how many seats remaining.

In [42]:
std_marks_data=pd.read_csv('studentdetails.csv')
not_admitted=pd.DataFrame(columns=std_marks_data.columns)
clg_data=pd.read_csv('Remaining_Seats.csv')
filled_seats=pd.DataFrame(columns=clg_data.columns,index=clg_data.index)
filled_seats['CollegeCode']=clg_data['CollegeCode']
filled_seats.fillna(0,inplace=True)
remaining_seats=clg_data
sortData=pd.read_csv('rank_not_admitted.csv')

total_students=sortData.index.size

final_not_admitted=[]
choice_data=pd.read_csv('choice_filling_not_admitted.csv')
for i in range(total_students): #this will loop through all student and give them admission according to their choice
        tempstd=sortData['Enrollmentno'][i]
        temp_choice=choice_data.loc[choice_data['Enrollmentno']==tempstd]
        temp_index=choice_data.loc[choice_data['Enrollmentno']==tempstd].index.tolist()[0]
        for j in range(10):
            clg,branch=temp_choice['choice'+str(j+1)][temp_index].split('-')  
           
            if int(remaining_seats.loc[remaining_seats['CollegeCode']==clg][branch])>0:
                remaining_seats.loc[remaining_seats.CollegeCode==clg,branch]-=1
                filled_seats.loc[filled_seats.CollegeCode==clg,branch]+=1
                admitted_students.loc[admission_count]=[tempstd,clg,branch,'FinalRound']
                admission_count+=1
                break
            if 0==1:
                pass
            elif j==9:
                final_not_admitted.append(tempstd)
            else:
                
                pass
admitted_students.to_csv('Admission_Details_All_Students.csv',index=False) #Admission details for all students
remaining_seats.to_csv('Final_Remaining_Seats.csv',index=False)    # College details of all vacant seats in each branch
if len(final_not_admitted)>0:
    print("Following Students didn't get admission in any college:")
    for i in range(len(final_not_admitted)):
        print(i+1,'.',final_not_admitted[i])
else:
    print("All students got admission in their favourable college!!!")
print("\n\nCollege details with remaining seats in each branch:")

remaining_seats    #printing remaining seat data of all college for each branch

All students got admission in their favourable college!!!


College details with remaining seats in each branch:


,CollegeCode,CSE,IT,ME,EE,EC,IC,CH,CIVIL,AUTO
0,C0001,240,120,0,0,180,155,0,60,155
1,C0002,0,60,0,35,0,180,240,60,60
2,C0003,95,60,215,180,95,0,155,180,0
3,C0004,120,60,0,60,0,180,60,120,0
4,C0005,180,95,60,240,0,180,180,105,10
5,C0006,70,120,240,0,180,155,60,60,60
6,C0007,60,60,120,35,95,120,0,180,60
7,C0008,60,240,155,0,180,120,215,120,0
8,C0009,60,240,120,130,70,190,95,155,240
9,C0010,60,190,0,240,0,35,60,180,60
